In [8]:
try:
    import tflite_runtime.interpreter as tflite
except ModuleNotFoundError:
    print("Did you install the TFLite Runtime? \
    https://github.com/ricardodeazambuja/libedgetpu-rpi0/releases/tag/rpi0_tflite_edgetpu")


EDGETPU_SHARED_LIB = 'libedgetpu.so.1'

In [9]:
def load_model(model_file):
    if "edgetpu" in model_file:
        print("Using Edge TPU...")
        #
        # EdgeTPU Accelerator
        #
        device = [] # I have only one USB accelerator...
        tflite_interpreter = tflite.Interpreter(model_path=model_file, 
                                         experimental_delegates=[tflite.load_delegate(EDGETPU_SHARED_LIB,{'device': device[0]} if device else {})])
    else:
        print("Using CPU...")
        tflite_interpreter = tflite.Interpreter(model_path=model_file)
        
    tflite_interpreter.allocate_tensors()
    input_details = tflite_interpreter.get_input_details()
    output_details = tflite_interpreter.get_output_details()
    
    return tflite_interpreter, input_details, output_details

In [10]:
class inference_for_single_image_tflite:
    def __init__(self, path_to_model):
        tflite_interpreter, input_details, output_details = load_model(path_to_model)
        self.interpreter = tflite_interpreter

        self.input_details = input_details
        self.output_details = output_details
        self.width = self.input_details[0]['shape'][2]
        self.height = self.input_details[0]['shape'][1]

    def __call__(self, image_np_expanded):
        self.interpreter.set_tensor(self.input_details[0]['index'], image_np_expanded)
        self.interpreter.invoke()

        return [self.interpreter.get_tensor(self.output_details[out_i]['index'])[0] for out_i in range(len(self.output_details))]

In [11]:
from PIL import Image
from PIL import ImageDraw
from IPython.display import display

            
def process_bboxes(image, boxes, classes, scores, threshold=0.5, 
                   labels={1:'mask', 2: 'no_mask', 3:'poor_mask'},
                   crop=False):
    colors = ((128, 255, 102), (102, 255, 255), (232, 123, 212))
    colors = dict(zip(labels.keys(),colors))
    image_width, image_height = image.size

    if not crop:
        draw = ImageDraw.Draw(image)
        # Visualization of the results of a detection.
        for i in range(len(boxes)):
            if scores[i] > threshold:
                ymin = int(max(1, (boxes[i][0] * image_height)))
                xmin = int(max(1, (boxes[i][1] * image_width)))
                ymax = int(min(image_height, (boxes[i][2] * image_height)))
                xmax = int(min(image_width, (boxes[i][3] * image_width)))
                draw.rectangle((xmin, ymin, xmax, ymax), width=3, outline=colors[int(classes[i])])
                text = f'{labels[int(classes[i])]} {scores[i]*100:1.2f}%'
                draw.text((xmin+3, ymin-10), text, fill=colors[int(classes[i])], width=2)
        return image
    else:
        for i in range(len(boxes)):
            if scores[i] >= threshold:
                box = boxes[i]
                ymin = int(max(1, (boxes[i][0] * image_height)))
                xmin = int(max(1, (boxes[i][1] * image_width)))
                ymax = int(min(image_height, (boxes[i][2] * image_height)))
                xmax = int(min(image_width, (boxes[i][3] * image_width)))
                return image.crop((xmin, ymin, xmax, ymax))

## Testing the models

In [4]:
!wget --no-clobber "https://images.canadagoose.com/image/upload/w_1333,c_scale,f_auto,q_auto:best/v1601577550/product-image/5558U_1073.jpg" -O mask_example1.jpg
!wget --no-clobber  "https://i.cbc.ca/1.5901311.1612993040!/fileImage/httpImage/image.jpg_gen/derivatives/original_780/face-recognition-test.jpg" -O mask_example2.jpg
!wget --no-clobber  "https://media.npr.org/assets/img/2021/01/29/_dsf1400_custom-01c8b25d39b298816ef6996981e39e40477fe7c6.jpg" -O mask_example3.jpg

File ‘mask_example1.jpg’ already there; not retrieving.
File ‘mask_example2.jpg’ already there; not retrieving.
File ‘mask_example3.jpg’ already there; not retrieving.


In [21]:
img_filename = "mask_example2.jpg"

In [22]:
# PATH_TO_TFLITE = "ssdlite_mobiledet_mask.tflite"
PATH_TO_TFLITE = "ssdlite_mobiledet_mask_edgetpu.tflite"

In [23]:
run_inference_for_single_image_tflite_ssd = inference_for_single_image_tflite(PATH_TO_TFLITE)

Using Edge TPU...


In [24]:
import numpy as np

image = Image.open(img_filename)
image = image.convert('RGB')
image_width, image_height = image.size

# tflite will not have the image resizing in the model
input_width = run_inference_for_single_image_tflite_ssd.width
input_height = run_inference_for_single_image_tflite_ssd.height

resized_image = image.resize((input_width, input_height))
np_image = np.asarray(resized_image)

image_np_expanded = np.expand_dims(np_image, axis=0)

# Actual detection.
boxes, classes, scores, _ = run_inference_for_single_image_tflite_ssd(image_np_expanded)

classes = [i+1 for i in classes] # class indices here start at 0

In [25]:
classes

[2.0, 1.0, 1.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]

In [26]:
scores

array([0.80078125, 0.72265625, 0.27734375, 0.19921875, 0.11328125,
       0.1015625 , 0.09375   , 0.09375   , 0.08203125, 0.08203125],
      dtype=float32)

In [ ]:
resized_image

In [ ]:
process_bboxes(resized_image.copy(), boxes, classes, scores, threshold=0.5)

In [29]:
import time

total_loops = 10
init_time = time.monotonic()
for i in range(total_loops):
    # Actual detection.
    boxes, classes, scores, _ = run_inference_for_single_image_tflite_ssd(image_np_expanded)
    
final_time = time.monotonic()
print(f"Total time: {final_time-init_time}")
print(f"FPS: {1/((final_time-init_time)/total_loops)}")

Total time: 0.5588072470000043
FPS: 17.895258255303055
